## Generate Grammar Items

This section of the notebook generates reads through every single map and creates mapping from a string to a column.

In [16]:
from map_generation.generate_image import pre_process_map
from tqdm import tqdm_notebook
import queue
import json
import os

In [17]:
TARGET_DIR = 'grammar/grammar_values/'
LEVELS_DIR = 'levels/csv_cleaned_maps'

In [18]:
columns = {}

In [19]:
files = os.listdir(LEVELS_DIR)
for file in tqdm_notebook(files):
    # read file
    f = open(os.path.join(LEVELS_DIR, file))
    lines = f.readlines()
    f.close()
    
    # convert lines into pre-processed
    for i in range(len(lines)):
        lines[i] = lines[i].strip()
    pre_process_map(lines)
    
    # convert lines to grammar
    for line in lines:
        line_str = ','.join(line)
        if line_str not in columns:
            columns[line_str] = 0
        
        columns[line_str] += 1

In [20]:
grammar = {}
grammar_reversed = {}

i = 0
for key in columns:
    grammar[i] = key
    grammar_reversed[key] = i
    i += 1

In [21]:
f = open(os.path.join(TARGET_DIR, 'grammar.json'), 'w')
f.write(json.dumps(grammar))
f.close()

f = open(os.path.join(TARGET_DIR, 'grammar_reversed.json'), 'w')
f.write(json.dumps(grammar_reversed))
f.close()

## Grammar Generator

This section will read through the maps again, but this time use the mappings created, above, to create a full grammar.

In [23]:
CLEANED_MAPS = 'levels/csv_cleaned_maps/'
MAX_SIZE = 5

In [24]:
grammars = [{} for i in range(MAX_SIZE)]

In [25]:
def convert_to_level_matrix(lvl_matrix):
    for i in range(len(lvl_matrix)):
        lvl_matrix[i] = str(grammar_reversed[lvl_matrix[i].strip()])

In [33]:
def convert_to_grammar(lvl_matrix, lvl_grammar, size):
    q = queue.Queue()
    for i in range(size):
        q.put('0')
    
    for col in lvl_matrix[size:]:
        grammar_value = ','.join(list(q.queue))
        col = ','.join(col)
        
        if grammar_value not in lvl_grammar:
            lvl_grammar[grammar_value] = {}
        
        if col not in lvl_grammar[grammar_value]:
            lvl_grammar[grammar_value][col] = 0
            
        lvl_grammar[grammar_value][col] += 1
        
        last_val = q.get()
        q.put(col)

In [34]:
files = os.listdir(CLEANED_MAPS)

for file in tqdm_notebook(files):
    f = open(os.path.join(CLEANED_MAPS, file), 'r')
    lines = f.readlines()
    f.close()
    
    for i in range(len(lines)):
        lines[i] = lines[i].strip()
    pre_process_map(lines)
    
    for size in range(MAX_SIZE):
        convert_to_grammar(lines, grammars[size], size+1)

|,:,:,:,:,:,:,:,:,:,:,:
0
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,=,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,=,=,:,:,:,:,:,:,:,:,:
|,=,:,:,:,:,:,:,:,:,:,:
|,=,=,=,:,:,:,:,:,:,:,:
|,=,=,:,:,:,:,:,:,:,:,:
|,=,=,=,=,:,:,:,:,:,:,:
|,=,=,=,:,:,:,:,:,:,:,:
|,:,:,:,=,:,:,:,:,:,:,:
|,=,=,=,=,:,:,:,:,:,:,:
|,:,:,:,=,zb,zt,:,:,:,:,:
|,:,:,:,=,:,:,:,:,:,:,:
|,:,:,:,=,:,:,:,:,:,:,:
|,:,:,:,=,zb,zt,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,=,:,:,:,:,:,:,:
|,k,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,c,:,:
|,k,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,

|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,pbl,pbl,ptl,el,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,pbr,pbr,ptr,er,:,:,:,:,:,:,:
|,pbl,pbl,ptl,el,:,:,:,:,:,:,:
|,k,:,:,:,:,:,:,:,:,:,:
|,pbr,pbr,ptr,er,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,k,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,pbl,ptl,el,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,pbr,ptr,er,:,:,:,:,:,:,:,:
|,pbl,ptl,el,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,pbr,ptr,er,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,zb,zt,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,zb,zt,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,

P,P,P,P,TM,:,:,:,:,:,:,:
TR,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,TL,:,:,:,:,:,:,:
:,:,:,:,TR,:,:,:,:,:,:,:
:,:,:,:,TL,:,:,:,:,:,:,:,P,P,P,P,TM,:,:,:,:,:,:,:
:,:,:,:,:,:,:,:,:,:,:,:
P,P,P,P,TM,:,:,:,:,:,:,:,:,:,:,:,TR,:,:,:,:,:,:,:
:,:,:,:,:,:,:,:,:,K,:,:
:,:,:,:,TR,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:
:,:,:,:,:,:,:,:,:,:,:,:
:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,K,:,:
:,:,:,:,:,:,:,TL,:,:,:,:
:,:,:,:,:,:,:,:,:,K,:,:,:,:,:,:,:,:,:,:,:,:,:,:
P,P,P,P,P,P,P,TM,g,:,:,:
:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,TL,:,:,:,:
P,P,P,P,P,P,P,TM,:,:,:,:
:,:,:,:,:,:,:,TL,:,:,:,:,P,P,P,P,P,P,P,TM,g,:,:,:
P,P,P,P,P,P,P,TM,:,:,:,:
P,P,P,P,P,P,P,TM,g,:,:,:,P,P,P,P,P,P,P,TM,:,:,:,:
P,P,P,P,P,P,P,TM,:,:,:,:
P,P,P,P,P,P,P,TM,:,:,:,:,P,P,P,P,P,P,P,TM,:,:,:,:
:,:,:,:,:,:,:,TR,:,:,:,:
P,P,P,P,P,P,P,TM,:,:,:,:,P,P,P,P,P,P,P,TM,:,:,:,:
:,:,:,:,:,:,:,:,:,:,:,:
P,P,P,P,P,P,P,TM,:,:,:,:,:,:,:,:,:,:,:,TR,:,:,:,:
:,:,:,:,:,:,:,:,:,:,:,:
:,:,:,:,:,:,:,TR,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:
:,:,:,:,:,:,:,:,:,:,:,:
:,:,:,:,:,:,:

|,:,:,:,:,:,:,:,:,:,:,:
|,g,:,:,:,:,:,:,:,:,:,:,|,g,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,g,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,R,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,R,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,

|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,#,:,:,:
|,:,:,:,:,:,:,:,#,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,#,:,:,:,|,:,:,:,:,:,:,:,#,:,:,:
|,:,:,:,#,:,:,:,?,:,:,:
|,:,:,:,:,:,:,:,#,:,:,:,|,:,:,:,:,:,:,:,#,:,:,:,|,:,:,:,:,:,:,:,#,:,:,:
|,g,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,#,:,:,:,|,:,:,:,:,:,:,:,#,:,:,:,|,:,:,:,#,:,:,:,?,:,:,:
|,g,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,#,:,:,:,|,:,:,:,#,:,:,:,?,:,:,:,|,g,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,#,:,:,:,?,:,:,:,|,g,:,:,:,:,:,:,:,:,:,:,|,g,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,g,:,:,:,:,:,:,:,:,:,:,|,g,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,g,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,#,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,#,s,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,#,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,#,:,:,:,

...

TM,:,:,:,:,:,:,:,:,:,:,:,TM,:,:,:,:,:,:,:,:,:,:,:,TM,:,:,:,:,:,:,:,:,:,:,:
TM,:,:,:,:,:,:,:,:,:,:,:
TM,:,:,:,:,:,:,:,:,:,:,:,TM,:,:,:,:,:,:,:,:,:,:,:,TM,:,:,:,:,:,:,:,:,:,:,:
TM,:,:,:,:,:,:,:,:,:,:,:
TM,:,:,:,:,:,:,:,:,:,:,:,TM,:,:,:,:,:,:,:,:,:,:,:,TM,:,:,:,:,:,:,:,:,:,:,:
TM,:,:,:,:,:,:,:,:,:,:,:
TM,:,:,:,:,:,:,:,:,:,:,:,TM,:,:,:,:,:,:,:,:,:,:,:,TM,:,:,:,:,:,:,:,:,:,:,:
TR,:,:,:,:,:,:,:,:,:,:,:
TM,:,:,:,:,:,:,:,:,:,:,:,TM,:,:,:,:,:,:,:,:,:,:,:,TM,:,:,:,:,:,:,:,:,:,:,:
:,:,:,:,:,:,:,:,:,:,:,:
TM,:,:,:,:,:,:,:,:,:,:,:,TM,:,:,:,:,:,:,:,:,:,:,:,TR,:,:,:,:,:,:,:,:,:,:,:
:,:,:,:,:,:,:,:,:,:,:,:
TM,:,:,:,:,:,:,:,:,:,:,:,TR,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
TR,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:
|,=,:,:,:,:,:,:,:,:,:,:
:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,=,=,:,:,:,:,:,:,:,:,:
:,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,=,:,:,:,:,:,:,:,:,:,:
|,=,=,=,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:

|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,=,=,=,=,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,=,=,=,=,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,=,=,=,=,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,=,=,=,=,:,:,:,:,:,:,:,|,=,=,=,=,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,=,=,=,=,:,:,:,:,:,:,:,|,=,=,=,=,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,=,=,=,=,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,#,:,:,:,#,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,#,:,:,:,#,m,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,

|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,#,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,sb,st,:,:,:,:,:,#,m,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,#,:,:,:
:,:,:,:,:,:,:,:,#,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,#,:,:,:,|,sb,st,:,:,:,:,:,#,m,:,:
|,:,:,:,:,:,:,:,#,:,:,:
|,:,:,:,:,:,:,:,#,:,:,:,|,sb,st,:,:,:,:,:,#,m,:,:,:,:,:,:,:,:,:,:,#,:,:,:
|,:,:,:,:,:,:,:,#,:,:,:
|,sb,st,:,:,:,:,:,#,m,:,:,:,:,:,:,:,:,:,:,#,:,:,:,|,:,:,:,:,:,:,:,#,:,:,:
|,:,:,:,:,:,:,:,#,:,:,:
:,:,:,:,:,:,:,:,#,:,:,:,|,:,:,:,:,:,:,:,#,:,:,:,|,:,:,:,:,:,:,:,#,:,:,:
|,:,:,:,:,:,:,:,#,:,:,:
|,:,:,:,:,:,:,:,#,:,:,:,|,:,:,:,:,:,:,:,#,:,:,:,|,:,:,:,:,:,:,:,#,:,:,:
:,:,:,:,:,:,:,:,#,:,:,:
|,:,:,:,:,:,:,:,#,:,:,:,|,:,:,:,:,:,:,:,#,:,:,:,|,:,:,:,:,:,:,:,#,:,:,:
|,:,:,:,:,:,:,:,#,:,:,:
|,:,:,:,:,:,:,:,#,:,:,:,|,:,:,:,:,:,:,:,#,:,:,:,:,:,:,:,:,:,:,:,#,:,:,:
:,:,:,:,:,:,:,:,#,:,:,:
|,:,:,:,

|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,g,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,g,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,g,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,pbl,pbl,pbl,ptl,el,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,pbr,pbr,pbr,ptr,er,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,pbl,pbl,pbl,ptl,el,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,pbl,pbl,pbl,ptl,el,:,:,:,:,:,:,|,pbr,pbr,pbr,ptr,er,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:

|,pbl,pbl,ptl,el,:,:,:,:,:,:,:,|,pbr,pbr,ptr,er,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,pbr,pbr,ptr,er,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,pbl,ptl,el,:,:,:,:,:,:,:,:
|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:,|,:,:,:,:,:,:,:,:,:,:,:
|,pbr,ptr,er,:,:,:,:,:,:,:,:
|,:,:,:,:

In [ ]:
for i in tqdm_notebook(range(len(grammars))):
    size_g = grammars[i]
    for g in size_g:
        total = 0
        for key in size_g[g]:
            total += size_g[g][key]

        for key in size_g[g]:
            size_g[g][key] /= float(total)

In [ ]:
for i in range(len(grammars)):
    f = open('grammar/grammar_values/%s_grammar.json' % (str(i + 1)), 'w')
    f.write(json.dumps(grammars[i]))
    f.close()